## Template for the usage of the project

Small example on basis of the first task to show the usage and test/debug the project.

### Environment Setup



In [ ]:
import os
from pathlib import Path as P

import numpy as np

import torch
import torch.nn as nn

import optuna

###--- Jupyter Environment Parameter ---###
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

CONFIG_PATH = P('')

EXPERIMENT_NAME = 'template'
RUN_NAME = 'test_01'

# Set required environment variables
os.environ['ACTIVATE_EXP'] = EXPERIMENT_NAME
os.environ['ACTIVE_RUN'] = RUN_NAME


### Hyperparameters

In [ ]:
###--- Model Configuration ---###
MODEL_CONFIG = {
                    'run_name' : 'test',
                    'model' : 'mlp',
                    'num_epochs' : 10,
                    'num_iterations': 0,       # 0 indicates that in each epoch the complete dataset is traversed. This value should be overwritter in code.
                    'dataset': {
                            'name': 'svhn',
                            'train_size': 73257,
                            'val_size': 6000,
                            'test_size': 26032,
                            'train_shuffle': True,
                            'eval_shuffle': False,
                            'drop_last': True
                    },
                    'batch_size' : 64,
                    'loss': {
                            'type': 'CrossEntropy'
                    },
                    'eval_batch_size': 32,
                    'eval_frequency' : 2,
                    'pre_processing': {
                            'flatten': True,
                            'flatten_only_img_sizes': True,
                            'rgb2gray': True,
                            'lbl_oneHot': True
                    },
                    'layers': [
                            {
                            'type': 'linear',
                            'dimension': 518,
                            'activation': 'relu'
                            },
                            {
                            'type': 'linear',
                            'dimension': 128,
                            'activation': 'relu'
                            },
                            {
                            'type': 'linear',
                            'dimension': 64,
                            'activation': 'relu'
                            },
                            {
                            'type': 'linear',
                            'dimension': 32,
                            'activation': 'relu'
                            },
                            {
                            'type': 'linear',
                            'dimension': 10,
                            'activation': 'softmax'
                            }

                    ],
                    'input_dim': 1024,
                    'output_dim': 10,
                    'optimizer': {
                            'type': 'Adam',
                            'learning_rate': 0.001,
                            'betas': [0.9, 0.99],
                            'eps': 0.00000001
                    },
                    'evaluation': {
                            'metrics': ['accuracy', 'precision', 'precision_per_class', 'recall', 'recall_per_class'],
                            'classes': [],   # Has to be defined prior to training from the dataset meta information.
                    },
                    'save_frequency': 2,
                    'random_seed': 42

                    }
###--- Hyperparameter Optimization Configuration ---###
OPTIMIZATION_CONFIG = {}



### Data Loading ###

Load data and print example images.